In [5]:
import re
import pandas as pd
import numpy as np
from iso3166 import countries
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import strip_numeric, strip_non_alphanum
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [12]:
glassdoor = pd.read_csv("/users/alinanary/downloads/glassdoor.csv")
country_codes = pd.read_csv("/users/alinanary/downloads/country_names_2_digit_codes.csv", index_col=1)

In [13]:
print(glassdoor.shape)
glassdoor.head()

(165290, 163)


,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,salary.currency.displayName,salary.currency.id,salary.currency.name,salary.currency.negativeTemplate,salary.currency.new,salary.currency.positiveTemplate,salary.currency.symbol,salary.lastSalaryDate,salary.salaries,wwfu
0,0.0,2801.0,2801.0,0,NaN,2801,-1,2183810,Modus Group,501-1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2483.0,NaN
1,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,United Kingdom Pound (GBP),2.0,United Kingdom Pound,$(#),False,$#,&pound;,2019-10-14T06:05:58,2484.0,NaN
2,0.0,2803.0,2803.0,0,NaN,2803,-1,1043373,Immobiliare.it,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2485.0,NaN
3,0.0,2804.0,2804.0,0,NaN,2804,-1,2226886,Codup,1-50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2486.0,NaN
4,0.0,2805.0,2805.0,0,NaN,2805,-1,851555,Markelytics Solutions,201-500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2487.0,NaN


In [14]:
print(glassdoor.columns.values)

['benefits.benefitRatingDecimal' 'benefits.comments' 'benefits.highlights'
 'benefits.numRatings' 'benefits.employerSummary' 'breadCrumbs'
 'gaTrackerData.category' 'gaTrackerData.empId' 'gaTrackerData.empName'
 'gaTrackerData.empSize' 'gaTrackerData.expired' 'gaTrackerData.industry'
 'gaTrackerData.industryId' 'gaTrackerData.jobId.long'
 'gaTrackerData.jobId.int' 'gaTrackerData.jobTitle'
 'gaTrackerData.location' 'gaTrackerData.locationId'
 'gaTrackerData.locationType' 'gaTrackerData.pageRequestGuid.guid'
 'gaTrackerData.pageRequestGuid.guidValid'
 'gaTrackerData.pageRequestGuid.part1'
 'gaTrackerData.pageRequestGuid.part2' 'gaTrackerData.sector'
 'gaTrackerData.sectorId'
 'gaTrackerData.profileConversionTrackingParams.trackingCAT'
 'gaTrackerData.profileConversionTrackingParams.trackingSRC'
 'gaTrackerData.profileConversionTrackingParams.trackingXSP'
 'gaTrackerData.jobViewTrackingResult.jobViewDisplayTimeMillis'
 'gaTrackerData.jobViewTrackingResult.requiresTracking'
 'gaTrackerData

In [15]:
print(list(country_codes.index))

['AF', 'AX', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BV', 'BR', 'IO', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA', 'CV', 'KY', 'CF', 'TD', 'CL', 'CN', 'CX', 'CC', 'CO', 'KM', 'CG', 'CD', 'CK', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'TF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG', 'GN', 'GW', 'GY', 'HT', 'HM', 'VA', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE', 'KI', 'KP', 'KR', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MO', 'MK', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR', 'MU', 'YT', 'MX', 'FM', 'MD', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', nan, 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'NF', 'MP', 'NO', 'OM'

In [16]:
country_list = glassdoor["map.country"].dropna()
country_list.unique()

array(['Poland', 'Italy', 'PK', 'IN', 'IL', 'FR', 'ID', 'HK', 'US', 'AE',
       'BE', 'South Korea', 'France', 'United Kingdom', 'Malaysia', 'ZM',
       'NG', 'SG', 'ES', 'CA', 'Austria', 'Ukraine', 'AT', 'RO', 'DE',
       'SA', 'IT', 'Germany', 'MY', 'AR', 'ZA', 'UA', 'PL', 'Israel',
       'Thailand', 'TH', 'New Zealand', 'Switzerland', 'CHE',
       'Netherlands', 'Hong Kong', 'LU', 'Belgium', 'Nigeria',
       'Indonesia', 'UK', 'Saudi Arabia', 'GB', 'IE', 'GH', 'UG', 'EG',
       'Australia', 'Nederland', 'AU', 'Mexico', 'India', 'TW', 'Sweden',
       'PH', 'SK', 'CN', 'Lithuania', 'MM', 'NL', 'Saint Kitts And Nevis',
       'CZ', 'Turkey', 'ro', 'LT', 'UZ', 'Finland', 'JP', 'United States',
       'Russia', 'USA', 'RU', 'NZ', 'Portugal', 'Kingdom', 'Spain', 'CH',
       'DK', 'TR', 'South Africa', 'Serbia', 'Zimbabwe', 'Kenya', 'Japan',
       'Slovakia', 'Singapore', 'BR', 'China', 'QA', 'Hungary', 'BY',
       'Romania', 'nl', 'Taiwan', 'Greece', 'MX', 'Ireland',
       'UN

In [17]:
clist = []
for item in country_list:
    temp = item.split(" - ")
    temp = temp[0].split(" [")
    t = re.sub(r"[^a-zA-Z]+",' ',temp[0])
    t = t.strip()
    clist.append(t)

pd.Series(clist).unique()

array(['Poland', 'Italy', 'PK', 'IN', 'IL', 'FR', 'ID', 'HK', 'US', 'AE',
       'BE', 'South Korea', 'France', 'United Kingdom', 'Malaysia', 'ZM',
       'NG', 'SG', 'ES', 'CA', 'Austria', 'Ukraine', 'AT', 'RO', 'DE',
       'SA', 'IT', 'Germany', 'MY', 'AR', 'ZA', 'UA', 'PL', 'Israel',
       'Thailand', 'TH', 'New Zealand', 'Switzerland', 'CHE',
       'Netherlands', 'Hong Kong', 'LU', 'Belgium', 'Nigeria',
       'Indonesia', 'UK', 'Saudi Arabia', 'GB', 'IE', 'GH', 'UG', 'EG',
       'Australia', 'Nederland', 'AU', 'Mexico', 'India', 'TW', 'Sweden',
       'PH', 'SK', 'CN', 'Lithuania', 'MM', 'NL', 'Saint Kitts And Nevis',
       'CZ', 'Turkey', 'ro', 'LT', 'UZ', 'Finland', 'JP', 'United States',
       'Russia', 'USA', 'RU', 'NZ', 'Portugal', 'Kingdom', 'Spain', 'CH',
       'DK', 'TR', 'South Africa', 'Serbia', 'Zimbabwe', 'Kenya', 'Japan',
       'Slovakia', 'Singapore', 'BR', 'China', 'QA', 'Hungary', 'BY',
       'Romania', 'nl', 'Taiwan', 'Greece', 'MX', 'Ireland',
       'UN

In [18]:
country_list_new = pd.Series([country_codes.loc[x,"Name"] if x in country_codes.index else x for x in clist])
country_list_new.dropna()
print(country_list_new.unique())

['Poland' 'Italy' 'Pakistan' 'India' 'Israel' 'France' 'Indonesia'
 'Hong Kong' 'United States' 'United Arab Emirates' 'Belgium'
 'South Korea' 'United Kingdom' 'Malaysia' 'Zambia' 'Nigeria' 'Singapore'
 'Spain' 'Canada' 'Austria' 'Ukraine' 'Romania' 'Germany' 'Saudi Arabia'
 'Argentina' 'South Africa' 'Thailand' 'New Zealand' 'Switzerland' 'CHE'
 'Netherlands' 'Luxembourg' 'UK' 'Ireland' 'Ghana' 'Uganda' 'Egypt'
 'Australia' 'Nederland' 'Mexico' 'Taiwan, Province of China' 'Sweden'
 'Philippines' 'Slovakia' 'China' 'Lithuania' 'Myanmar'
 'Saint Kitts And Nevis' 'Czech Republic' 'Turkey' 'ro' 'Uzbekistan'
 'Finland' 'Japan' 'Russia' 'USA' 'Russian Federation' 'Portugal'
 'Kingdom' 'Denmark' 'Serbia' 'Zimbabwe' 'Kenya' 'Brazil' 'Qatar'
 'Hungary' 'Belarus' 'nl' 'Taiwan' 'Greece' 'UNITED STATES'
 'Korea, Republic of' 'at' 'Estonia' 'Viet Nam' 'Cyprus' 'SINGAPORE'
 'Chile' 'Tanzania' 'Panama' 'Uruguay' 'Costa Rica' 'Bulgaria' 'Jordan'
 'Bangladesh' 'Iceland' 'JAMAICA' 'ng' 'vn' 'Albania' 

In [19]:
def rename(country):
    try:
        return countries.get(country).name
    except:
        return (np.nan)

country_list_new = pd.Series(clist)
rows_before = country_list_new.shape[0]
country_list_new = country_list_new.apply(rename)
country_list_new = country_list_new.dropna()
rows_after = country_list_new.shape[0]
print("Samples after: {}".format(rows_before))
print("Samples earlier: {}".format(rows_after))
print("Lost {} samples after converting which is {:.2f}% of the data\n".format((rows_before-rows_after),
                                                                    ((rows_before-rows_after)/rows_before)*100))

print(country_list_new.unique())

Samples after: 115262
Samples earlier: 110061
Lost 5201 samples after converting which is 4.51% of the data

['Poland' 'Italy' 'Pakistan' 'India' 'Israel' 'France' 'Indonesia'
 'Hong Kong' 'United States of America' 'United Arab Emirates' 'Belgium'
 'Malaysia' 'Zambia' 'Nigeria' 'Singapore' 'Spain' 'Canada' 'Austria'
 'Ukraine' 'Romania' 'Germany' 'Saudi Arabia' 'Argentina' 'South Africa'
 'Thailand' 'New Zealand' 'Switzerland' 'Netherlands' 'Luxembourg'
 'United Kingdom of Great Britain and Northern Ireland' 'Ireland' 'Ghana'
 'Uganda' 'Egypt' 'Australia' 'Mexico' 'Taiwan, Province of China'
 'Sweden' 'Philippines' 'Slovakia' 'China' 'Lithuania' 'Myanmar'
 'Saint Kitts and Nevis' 'Czechia' 'Uzbekistan' 'Finland' 'Japan'
 'Russian Federation' 'Portugal' 'Denmark' 'Türkiye' 'Serbia' 'Zimbabwe'
 'Kenya' 'Brazil' 'Qatar' 'Hungary' 'Belarus' 'Greece'
 'Korea, Republic of' 'Estonia' 'Viet Nam' 'Cyprus' 'Chile' 'Panama'
 'Uruguay' 'Costa Rica' 'Bulgaria' 'Jordan' 'Bangladesh' 'Iceland'
 'Jam

In [30]:
country_company_counts = glassdoor.groupby(['map.country', 'header.employerName']).size().reset_index(name='job_count')

# Sort the DataFrame in descending order by job count
country_company_counts = country_company_counts.sort_values(by=['map.country', 'job_count'], ascending=[True, False])

# Filter the data to only include full country names from country_list_new.unique()
country_list_filtered = country_list_new.unique()
top_companies_by_country = country_company_counts[country_company_counts['map.country'].isin(country_list_filtered)]

# Get the top 100 companies for each country
top_companies_by_country = top_companies_by_country.groupby('map.country').head(100)

# Display the DataFrame
print(top_companies_by_country)

       map.country       header.employerName  job_count
2074   Afghanistan        CACI International          7
2075   Afghanistan                   CopaSAT          1
2076   Afghanistan  Planate Management Group          1
2078       Albania                Teach Away          5
2077       Albania                  Red Bull          1
...            ...                       ...        ...
33637       Zambia                   Sandvik          1
33640     Zimbabwe            Econet Careers          2
33641     Zimbabwe         Solidarity Center          2
33639     Zimbabwe                  Deloitte          1
33642     Zimbabwe         Turner & Townsend          1

[3860 rows x 3 columns]


In [34]:
output_file = "/users/alinanary/downloads/top_companies_by_country.csv"

top_companies_by_country.to_csv(output_file, index=False)


Data saved to /users/alinanary/downloads/top_companies_by_country.csv


In [35]:
filtered_companies_by_country = top_companies_by_country[top_companies_by_country['job_count'] > 3]

# Display the filtered DataFrame
print(filtered_companies_by_country)

                map.country        header.employerName  job_count
2074            Afghanistan         CACI International          7
2078                Albania                 Teach Away          5
2082                Algeria                   Marriott         19
2084                Algeria                Weatherford          4
2085              Argentina                  Accenture         15
...                     ...                        ...        ...
31726  United Arab Emirates              Duff & Phelps          5
31730  United Arab Emirates  Fairmont Hotels & Resorts          5
31729  United Arab Emirates                   Facebook          4
32904               Uruguay                  Trafilea          16
32956              Viet Nam                Atlas Copco          4

[1752 rows x 3 columns]


In [36]:
output_file = "/users/alinanary/downloads/filtered_companies_by_country.csv"

filtered_companies_by_country.to_csv(output_file, index=False)